[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/transformers/HuggingFace%20in%20Spark%20NLP%20-%20XlNet.ipynb)

## Import XLNet models from HuggingFace 🤗  into Spark NLP 🚀 

Let's keep in mind a few things before we start 😊 

- This feature is only available in `Spark NLP 3.1.2` and above. So please make sure you have upgraded to the latest Spark NLP release
- You can import models for XLNet from HuggingFace but they have to be compatible with `TensorFlow` and they have to be in `Fill Mask` category. Meaning, you cannot use XLNet models trained/fine-tuned on a specific task such as token/sequence classification.
- Some of the `XLNet` are larger than 2GB which is the limit in Spark NLP. Unfortunately, the models must be less than 2GB in size

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.4.1` version and Transformers on `4.6.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- XLNetTokenizer requires the `SentencePiece` library, so we install that as well

In [1]:
!pip install -q transformers==4.6.1 tensorflow==2.4.1 sentencepiece

     |████████████████████████████████| 2.3MB 7.8MB/s 
     |████████████████████████████████| 394.3MB 39kB/s 
     |████████████████████████████████| 1.2MB 44.4MB/s 
     |████████████████████████████████| 3.3MB 35.7MB/s 
     |████████████████████████████████| 901kB 40.5MB/s 
     |████████████████████████████████| 2.9MB 33.1MB/s 
     |████████████████████████████████| 471kB 58.8MB/s 
     |████████████████████████████████| 3.8MB 26.3MB/s 


- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [xlnet-base-cased](https://huggingface.co/xlnet-base-cased) model from HuggingFace as an example
- In addition to `TFXLNetModel` we also need to save the `XLNetTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [2]:
from transformers import XLNetTokenizer, TFXLNetModel
import tensorflow as tf

# xlnet-base-cased
MODEL_NAME = 'xlnet-base-cased'

XLNetTokenizer.from_pretrained(MODEL_NAME, return_tensors="pt").save_pretrained("./{}_tokenizer".format(MODEL_NAME))
TFXLNetModel.from_pretrained(MODEL_NAME).save_pretrained("./{}".format(MODEL_NAME), saved_model=True)

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

INFO:tensorflow:Assets written to: ./xlnet-base-cased/saved_model/1/assets


INFO:tensorflow:Assets written to: ./xlnet-base-cased/saved_model/1/assets


Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {MODEL_NAME}

total 456116
-rw-r--r-- 1 root root       881 Jul  7 10:35 config.json
drwxr-xr-x 3 root root      4096 Jul  7 10:35 saved_model
-rw-r--r-- 1 root root 467046720 Jul  7 10:36 tf_model.h5


In [4]:
!ls -l {MODEL_NAME}/saved_model/1

total 5908
drwxr-xr-x 2 root root    4096 Jul  7 10:35 assets
-rw-r--r-- 1 root root 6038897 Jul  7 10:35 saved_model.pb
drwxr-xr-x 2 root root    4096 Jul  7 10:35 variables


In [5]:
!ls -l {MODEL_NAME}_tokenizer

total 788
-rw-r--r-- 1 root root    290 Jul  7 10:34 special_tokens_map.json
-rw-r--r-- 1 root root 798011 Jul  7 10:34 spiece.model
-rw-r--r-- 1 root root    665 Jul  7 10:34 tokenizer_config.json


- as you can see, we need the SavedModel from `saved_model/1/` path
- we also be needing `spiece.model` file from the tokenizer
- all we need is to copy `spiece.model` file into `saved_model/1/assets` which Spark NLP will look for

In [6]:
# let's copy spiece.model file to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/spiece.model {MODEL_NAME}/saved_model/1/assets

## Import and Save XLNet in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `XlnetEmbeddings` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `XlnetEmbeddings` in runtime, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want! 
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.


In [9]:
from sparknlp.annotator import *

xlnet = XlnetEmbeddings.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
 .setInputCols(["sentence",'token'])\
 .setOutputCol("embeddings")\
 .setCaseSensitive(True)\
 .setDimension(768)\
 .setStorageRef('xlnet_base_cased') 

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [10]:
xlnet.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [11]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your XLNet model from HuggingFace 🤗 loaded and saved by Spark NLP 🚀 

In [12]:
! ls -l {MODEL_NAME}_spark_nlp

total 424544
drwxr-xr-x 3 root root      4096 Jul  7 10:58 fields
drwxr-xr-x 2 root root      4096 Jul  7 10:58 metadata
-rw-r--r-- 1 root root    798011 Jul  7 11:02 xlnet_spp
-rw-r--r-- 1 root root 433923984 Jul  7 11:02 xlnet_tensorflow


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny RoBERTa model 😊 

In [13]:
xlnet_loaded = XlnetEmbeddings.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["sentence",'token'])\
  .setOutputCol("embeddings")\
  .setCaseSensitive(False)

In [14]:
xlnet_loaded.getStorageRef()

'xlnet_base_cased'

That's it! You can now go wild and use hundreds of XLNet models from HuggingFace 🤗 in Spark NLP 🚀 
